In [ ]:
import functools
from ema_workbench import Scenario
from ema_workbench import Model, MultiprocessingEvaluator, Policy, Scenario

from ema_workbench.em_framework.evaluators import perform_experiments
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.util import ema_logging
import time
from problem_formulation import get_model_for_problem_formulation
from ema_workbench.analysis import pairs_plotting
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from ema_workbench.analysis import prim
from ema_workbench.analysis import dimensional_stacking

from ema_workbench.analysis import feature_scoring
from pandas.api.types import is_numeric_dtype, is_categorical_dtype
from sklearn import preprocessing
from ema_workbench.em_framework.optimization import (ArchiveLogger,
                                                     EpsilonProgress,
                                                     to_problem, epsilon_nondominated)

from ema_workbench import ScalarOutcome

In [ ]:
# # generic function for calculating domain criterion robustness metric
# def robustness(direction, threshold, data):
#     if direction == SMALLER:
#         return np.sum(data<=threshold)/data.shape[0]
#     else:
#         return np.sum(data>=threshold)/data.shape[0]
#
# SMALLER = 'SMALLER'
# LARGER = 'LARGER'
#
# # we use a functional programming trick with the generic robustness
# # function to create our actual robustness function for each
# # objective (see https://docs.python.org/3/library/functools.html#functools.partial)
# maxp = functools.partial(robustness, SMALLER, 0.75)
# inertia = functools.partial(robustness, LARGER, 0.6)

In [ ]:
df_scenarios= pd.read_csv('reference_scenarios.csv')

df_scenarios = (
    df_scenarios
    .rename(columns={"scenario.1": "scenario"})  # als hij zo heet
    .set_index("scenario")                        # zet de kolom als index
)

candidate_scenarios = [
    Scenario(f"scenario_{idx}", **row.to_dict())
    for idx, row in df_scenarios.iterrows()
]

In [ ]:
# lees de CSV in met de gevonden policies vanuit de MORDM
policies_df = pd.read_csv("candidate_policies.csv")

# zet elke rij om in een Policy-object
candidate_policies = [
    Policy(f"cand_{i}", **row.to_dict())
    for i, row in policies_df.iterrows()

    ]

In [ ]:
if __name__ == "__main__":
    model, _ = get_model_for_problem_formulation(3)
    scenarios=10
    policies=4
    with MultiprocessingEvaluator(model, n_processes=-1) as evaluator:
        results = evaluator.perform_experiments(scenarios=scenarios, policies=candidate_policies )

        experiments, outcomes = results

In [ ]:
convergence_metrics = [EpsilonProgress()]


with MultiprocessingEvaluator(model) as evaluator:
    results, convergence = evaluator.robust_optimize(
        nfe=0,                            # no new search, just evaluate your eight policies
        searchover="policies",           # search space is your list of policies
        policies=candidate_policies,
        reference=candidate_scenarios,   # multi‐scenario evaluation
        convergence=[EpsilonProgress()], # track progress (optional)
    )
    results, convergence = evaluator.robust_optimize(robustness_functions, candidate_scenarios,
                                                     nfe=nfe, epsilons=[0.05,]*len(robustness_functions),
                                                     convergence=convergence_metrics,)

# 4) look at your results
#    - one row per policy
#    - columns give you robust performance metrics
import pandas as pd
df = pd.DataFrame(results)

# e.g. sort by 5th‐percentile of your worst‐case outcome (or whichever metric you care about)
df = df.sort_values(by="outcome_name__5%", ascending=False)

print(df.head())